#####Libraries
***

In [1]:
import sklearn
import seaborn as sns
import pandas as pd
import numpy as np
import seaborn as sns
!pip install yfinance --upgrade --no-cache-dir
import yfinance as yf
from pandas_datareader import data as pdr
yf.pdr_override()
# pip install TA-lib
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz

!tar -xzf ta-lib-0.4.0-src.tar.gz
%cd ta-lib/
!./configure --prefix=$HOME
!make
!make install

!TA_LIBRARY_PATH=~/lib TA_INCLUDE_PATH=~/include pip install ta-lib
import talib as ta
import matplotlib.pyplot as plt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 5.6 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
--2022-09-15 13:28:34--  http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
Resolving prdownloads.sourceforge.net (prdownloads.sourceforge.net)... 204.68.111.105
Connecting to prdownloads.sourceforge.net (prdownloads.sourceforge.net)|204.68.111.105|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz [following]
--2022-09-15 13:28:34--  http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz
Resolving downloads.sourceforge.net (downloads.sourceforge.net)... 204.68.111.105
Reusing existing connection to prdownlo

#####Load Dataset
***

In [66]:
#φτιάχνω τη λίστα με τα assets μου -> 50 μετοχές 
#διόρθωση : θα βγάλω τυχαία άλλες 4 μετοχές μαζί με την OGN επειδή δεν έχει το ίδιο μέγεθος με τις άλλες 
#βγάζω OGN ,WRK,HST,CE,ACN
assets= ["ABBV","AEP","AIZ","ALLE","AMAT","AMP","AMZN","AVB","AVY",   
  "AXP","BDX","BF-B","BMY","BR","CARR","CDW","CHTR","CNC",   
  "CNP","COP","CTAS","CZR","DG","DPZ","DRE","DXC","META","FTV",   
  "GOOG","GPC","HIG","JPM","KR","PG","PPL","PRU",   
  "PYPL","RE","ROL","ROST","UNH","URI","V","VRSK","XOM"]
  #άρα πρέπει να κατεβάσουμε όλα τα adj close για όλες τις τιμές όλων των μετοχών και να κάνουμε απόλυτο ranking και σε 5 κλασεις (targets)
r=pd.DataFrame(index=pdr.get_data_yahoo(assets[0],'2021-03-01','2022-07-01')['Adj Close'].index)
#παίρνω τιμές για ένα χρόνο και 3 μήνες
for i in assets:
  # print('i metoxi me onoma:',i,'exei megethos',pdr.get_data_yahoo(i,'2021-01-01','2022-04-01')['Adj Close'].count())
  temp= pdr.get_data_yahoo(i,'2021-03-01','2022-07-01')['Adj Close']
  temp=temp.dropna()
  r[i]=temp.values

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [67]:
#φτιάχνω ένα αντίγραφο με τις τιμές για να το έχω 
rd=pd.DataFrame(r)
rd

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-03-01,101.758369,72.935371,123.264771,109.629959,120.793564,223.396744,157.307007,169.428787,174.066833,136.349152,...,82.900154,273.630005,232.818298,33.455544,118.117958,326.841461,302.489990,214.501099,165.157501,52.391727
2021-03-02,101.232719,72.688164,123.596596,110.699760,116.100159,220.188354,154.726501,170.577332,173.244690,136.879959,...,83.274330,269.190002,232.127411,32.895821,115.247551,326.831726,293.420013,213.649567,164.504944,52.085182
2021-03-03,100.162682,72.564552,124.962952,110.071625,114.064705,218.140472,150.250000,173.620941,174.624710,140.163116,...,83.601746,255.059998,232.424881,31.874580,108.791565,325.102997,294.929993,212.738586,162.952637,52.503204
2021-03-04,99.167709,73.448792,125.001991,108.354057,106.950470,211.850479,148.878494,170.414642,168.527237,140.251556,...,82.853378,239.070007,229.978088,31.324680,109.741844,326.021057,288.600006,209.421524,159.828262,54.537560
2021-03-05,100.153282,75.017609,128.749710,111.210121,112.098396,216.784943,150.022995,170.031738,173.499161,144.822433,...,84.808487,239.050003,236.905853,32.345928,110.829277,339.000977,300.959991,213.293106,166.225327,56.599789
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-24,150.928238,92.870499,172.150925,99.599998,97.111656,248.137924,116.459999,193.575409,164.681396,145.605087,...,96.279518,77.680000,281.488770,35.017067,77.258453,494.089691,248.289993,205.146637,172.702087,86.063782
2022-06-27,151.720825,94.239746,172.300293,99.239998,97.361015,246.087448,113.220001,197.166199,166.274918,143.752029,...,95.824852,75.940002,281.329712,35.186615,76.072624,504.078339,251.990005,203.200089,173.810226,88.173286
2022-06-28,151.076859,94.338966,172.290344,98.400002,94.777618,242.633484,107.400002,195.470001,163.456360,141.659866,...,96.546379,71.820000,278.745331,34.418652,72.525093,506.849640,247.990005,197.769714,168.658813,90.619514


In [68]:
rd1= rd.pct_change()
rd1

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-03-02,-0.005166,-0.003389,0.002692,0.009758,-0.038855,-0.014362,-0.016404,0.006779,-0.004723,0.003893,...,0.004514,-0.016226,-0.002967,-0.016730,-0.024301,-0.000030,-0.029984,-0.003970,-0.003951,-0.005851
2021-03-03,-0.010570,-0.001701,0.011055,-0.005674,-0.017532,-0.009301,-0.028932,0.017843,0.007966,0.023986,...,0.003932,-0.052491,0.001281,-0.031045,-0.056018,-0.005289,0.005146,-0.004264,-0.009436,0.008026
2021-03-04,-0.009934,0.012186,0.000312,-0.015604,-0.062370,-0.028835,-0.009128,-0.018467,-0.034918,0.000631,...,-0.008952,-0.062691,-0.010527,-0.017252,0.008735,0.002824,-0.021463,-0.015592,-0.019174,0.038747
2021-03-05,0.009938,0.021359,0.029981,0.026359,0.048134,0.023292,0.007687,-0.002247,0.029502,0.032591,...,0.023597,-0.000084,0.030124,0.032602,0.009909,0.039813,0.042827,0.018487,0.040025,0.037813
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-24,0.019338,0.012877,0.020968,0.028819,0.043739,0.040616,0.035752,-0.006820,0.063412,0.031768,...,0.053195,0.052432,0.040260,0.028111,0.047986,-0.008343,0.052790,0.045108,0.011815,0.019833
2022-06-27,0.005251,0.014744,0.000868,-0.003614,0.002568,-0.008263,-0.027821,0.018550,0.009676,-0.012727,...,-0.004722,-0.022400,-0.000565,0.004842,-0.015349,0.020216,0.014902,-0.009489,0.006416,0.024511
2022-06-28,-0.004244,0.001053,-0.000058,-0.008464,-0.026534,-0.014036,-0.051404,-0.008603,-0.016951,-0.014554,...,0.007530,-0.054253,-0.009186,-0.021825,-0.046633,0.005498,-0.015874,-0.026724,-0.029638,0.027743


In [69]:
#prepare the ranks for next month (20 working days later)
rd1_next_month = rd1.shift(-22)
rd1_next_month

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-03-01,0.013391,-0.000354,-0.020317,0.002634,0.053960,-0.001503,0.012696,-0.015264,-0.008262,-0.011324,...,-0.010643,0.026634,-0.025483,0.014741,-0.003904,-0.003989,-0.011704,-0.000802,0.004206,-0.015170
2021-03-02,0.002772,0.009917,0.003739,0.016956,0.059281,0.012734,0.021628,0.012953,0.024067,0.022130,...,0.017673,0.019354,0.013317,0.011912,0.005004,-0.013438,0.007288,0.024229,0.017658,0.027942
2021-03-03,-0.021931,0.008768,0.003443,0.017847,0.010811,0.012191,0.020794,-0.004869,0.015313,0.010306,...,0.000431,0.014058,-0.000398,0.016078,0.020496,0.008473,-0.000543,0.010053,0.009677,-0.015508
2021-03-04,-0.007160,0.005563,0.004272,0.011305,-0.024537,-0.007764,-0.000902,0.002742,0.007122,0.000616,...,0.003127,0.008645,0.010239,-0.002826,0.004228,-0.015695,-0.018400,-0.001781,-0.003635,-0.002832
2021-03-05,-0.001613,-0.000922,0.015202,-0.001065,-0.002866,0.007021,0.017237,0.006863,-0.008892,0.006706,...,0.009781,0.009519,0.008597,-0.022953,-0.000729,0.003485,-0.010540,0.002836,-0.004975,0.004437
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [70]:
#first: ranks assigned in order they appear in the array
rd2=rd1_next_month.rank(axis=1,method='first')
rd2=rd2.dropna()
rd2.astype(int)
jk=rd2.copy()
jk.astype(int)

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-03-01,41,28,5,31,45,25,40,7,19,14,...,16,44,2,42,23,22,13,27,34,8
2021-03-02,13,21,15,29,45,24,36,25,41,37,...,32,33,26,23,16,3,17,42,31,43
2021-03-03,2,18,13,37,22,24,39,5,33,21,...,11,30,9,34,38,17,7,20,19,3
2021-03-04,14,36,31,41,1,12,22,28,38,26,...,29,39,40,19,30,6,5,21,16,18
2021-03-05,14,18,42,17,11,35,44,34,4,33,...,39,38,37,1,19,27,2,26,8,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-23,13,6,15,20,33,31,28,3,43,23,...,41,39,30,19,36,1,40,34,5,14
2022-05-24,30,39,20,17,23,10,1,41,36,6,...,14,2,19,27,5,43,40,8,32,45
2022-05-25,36,40,39,31,11,23,3,30,17,22,...,43,1,29,15,4,42,20,10,9,44


In [71]:
def rank_data(df1):
  for i in range(len(df1.index)):
    d=df1.loc[df1.index[i]]  #της κάθε μέρας d[1] δίνει μόνο τις τιμές 
    #print(d)
    d=d.sort_values()
    d[0:9]=1
    d[9:18]=2
    d[18:27]=3
    d[27:36]=4
    d[36:45]=5
    df1.loc[df1.index[i]]=d
    #print(df1.loc[df1.index[i]])
    #print(d)
  df1=df1.astype(int)
  return df1
r5=rd2.copy()
rd3=rank_data(r5)
rd3

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-03-01,5,4,1,4,5,3,5,1,3,2,...,2,5,1,5,3,3,2,3,4,1
2021-03-02,2,3,2,4,5,3,4,3,5,5,...,4,4,3,3,2,1,2,5,4,5
2021-03-03,1,2,2,5,3,3,5,1,4,3,...,2,4,1,4,5,2,1,3,3,1
2021-03-04,2,4,4,5,1,2,3,4,5,3,...,4,5,5,3,4,1,1,3,2,2
2021-03-05,2,2,5,2,2,4,5,4,1,4,...,5,5,5,1,3,3,1,3,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-23,2,1,2,3,4,4,4,1,5,3,...,5,5,4,3,4,1,5,4,1,2
2022-05-24,4,5,3,2,3,2,1,5,4,1,...,2,1,3,3,1,5,5,1,4,5
2022-05-25,4,5,5,4,2,3,1,4,2,3,...,5,1,4,2,1,5,3,2,1,5


In [72]:
#for absolute rank feature to use 
rd2_feature=rd1.rank(axis=1,method='first')
rd2_feature=rd2_feature.dropna()
rd2_feature.astype(int)
jk_feature=rd2_feature.copy()
jk_feature.astype(int)

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-03-02,25,30,39,45,1,12,10,42,26,40,...,41,11,33,9,4,37,3,28,29,21
2021-03-03,16,28,36,24,11,18,6,39,33,43,...,31,2,30,4,1,25,32,27,17,34
2021-03-04,24,40,32,19,2,6,25,16,5,33,...,26,1,23,17,37,34,11,20,12,45
2021-03-05,8,16,28,23,43,19,6,1,27,32,...,20,3,29,33,7,37,41,13,39,34
2021-03-08,9,34,28,41,1,22,7,43,42,33,...,38,2,29,8,45,24,19,37,35,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-24,13,6,15,20,33,31,28,3,43,23,...,41,39,30,19,36,1,40,34,5,14
2022-06-27,30,39,20,17,23,10,1,41,36,6,...,14,2,19,27,5,43,40,8,32,45
2022-06-28,36,40,39,31,11,23,3,30,17,22,...,43,1,29,15,4,42,20,10,9,44


In [73]:
r5_next_month=rd2_feature.copy()
rd3_next_month=rank_data(r5_next_month)
rd3_next_month

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-03-02,3,4,5,5,1,2,2,5,3,5,...,5,2,4,1,1,5,1,4,4,3
2021-03-03,2,4,4,3,2,2,1,5,4,5,...,4,1,4,1,1,3,4,3,2,4
2021-03-04,3,5,4,3,1,1,3,2,1,4,...,3,1,3,2,5,4,2,3,2,5
2021-03-05,1,2,4,3,5,3,1,1,3,4,...,3,1,4,4,1,5,5,2,5,4
2021-03-08,1,4,4,5,1,3,1,5,5,4,...,5,1,4,1,5,3,3,5,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-24,2,1,2,3,4,4,4,1,5,3,...,5,5,4,3,4,1,5,4,1,2
2022-06-27,4,5,3,2,3,2,1,5,4,1,...,2,1,3,3,1,5,5,1,4,5
2022-06-28,4,5,5,4,2,3,1,4,2,3,...,5,1,4,2,1,5,3,2,1,5


In [117]:
rd3_next_month.loc[rd3_next_month.index=='2022-05-27']

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2022-05-27,1,1,2,4,5,2,5,4,4,4,...,3,5,3,1,1,1,5,3,4,1


#####Functions
***

In [75]:
def RPS_calculation(hist_data, submission, asset_no=44):
    # print('check')
    if hist_data.shape[0]<=asset_no:
        return np.nan    
    # print('check')
    asset_id = pd.unique(hist_data.symbol)
    
    for i in range(len(pd.unique(hist_data.date))):
        if len(hist_data[hist_data.date == pd.unique(hist_data.date)[i]])<len(asset_id):
            for asset in [x for x in asset_id if x not in hist_data[hist_data.date == pd.unique(hist_data.date)[i]].symbol.values]:
                right_price = hist_data[hist_data.symbol==asset].sort_values(by='date')
                right_price = right_price[right_price.date <= pd.unique(hist_data.date)[i]]
                right_price = right_price.price.iloc[-1]
                hist_data = hist_data.append({'date' : pd.unique(hist_data.date)[i],
                                               'symbol' : asset,
                                               'price' : right_price}, ignore_index=True)

    #Compute percentage returns
    asset_id = sorted(asset_id) 

    #Compute percentage returns
    returns = pd.DataFrame(columns = ["ID", "Return"])
    
    min_date = min(hist_data.date)
    max_date = max(hist_data.date)
    # max_date = np.max(hist_data.date)
    
    for i in range(0,len(asset_id)):
        temp = hist_data.loc[hist_data.symbol==asset_id[i]]
        
        open_price = float(temp.loc[temp.date==min_date].price)
        close_price = float(temp.loc[temp.date==max_date].price)
        
        returns = returns.append({'ID': temp.symbol.iloc[0], 
                                'Return': (close_price - open_price)/open_price}, ignore_index=True)

    #Define the relevant position of each asset
    ranking = pd.DataFrame(columns=["ID", "Position", "Return"])
    ranking.ID = list(asset_id)
    ranking.Return = returns.Return
    ranking.Position = ranking.Return.rank(method = 'min')

    #Handle Ties
    Series_per_position = pd.DataFrame(columns=["Position","Series", "Rank", "Rank1", "Rank2","Rank3", "Rank4", "Rank5"])
    Series_per_position.Position = list(pd.unique(ranking.Position.sort_values(ascending=True)))
    temp = ranking.Position.value_counts()
    temp = pd.DataFrame(zip(temp.index, temp), columns = ["Rank", "Occurencies"])
    temp = temp.sort_values(by = ["Rank"],ascending=True)
    Series_per_position.Series = list(temp.Occurencies)
    Series_per_position

    total_ranks = Series_per_position.Position.values[-1]
    for i in range(0,Series_per_position.shape[0]):
    
        start_p = Series_per_position.Position[i]
        end_p = Series_per_position.Position[i] + Series_per_position.Series[i]
        temp = pd.DataFrame(columns = ["Position","Rank", "Rank1", "Rank2", "Rank3", "Rank4","Rank5"])
        temp.Position = list(range(int(start_p),int(end_p)))

        if(temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))].assign(Rank=1)
            temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))].assign(Rank1=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))].assign(Rank=2)
            temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))].assign(Rank2=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))].assign(Rank=3)
            temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))].assign(Rank3=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))].assign(Rank=4)
            temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))].assign(Rank4=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))].assign(Rank=5)
            temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))].assign(Rank5=1.0)
        temp = temp.fillna(0)
        Series_per_position.iloc[i,2:Series_per_position.shape[1]] = temp.mean(axis = 0).iloc[1:temp.shape[1]]

    Series_per_position = Series_per_position.drop('Series', axis = 1)
    ranking = pd.merge(ranking,Series_per_position, on = "Position")
    ranking = ranking[["ID", "Return", "Position", "Rank", "Rank1", "Rank2", "Rank3", "Rank4", "Rank5"]]
    ranking = ranking.sort_values(["Position"])
    # print(ranking)
    #Evaluate submission
    rps_sub = []
    #for aid in list((pd.unique(ranking.ID))):
    for aid in asset_id:

        target = np.cumsum(ranking.loc[ranking.ID==aid].iloc[:,4:9].values).tolist()
        frc = np.cumsum(submission.loc[submission.ID==aid].iloc[:,1:6].values).tolist()
        rps_sub.append(np.mean([(a - b)**2 for a, b in zip(target, frc)]))
    submission["RPS"] = rps_sub
    
    output = {'RPS' : np.mean(rps_sub),
              'details' : submission}
    
    return(output)

In [76]:
def create_historical(prob):
  prob=prob.rename_axis("symbol", axis="columns")
  prob=prob.T
  prob['date']=prob.columns.date[0]
  prob.columns = ['price', 'date']
  prob=prob.reset_index()
  prob=prob[['symbol','date','price']]
  return prob

In [77]:
def predictions(model,assets,stock,k):
  prob=dict()

  for j in assets:
      tmp=[]
      df=stock[j]
      df1=df.dropna()
      X= df1.iloc[:,5:k]
      # print(X) #it has all the days
      y= df1['Rank']
      X_train= X.iloc[:-23] #-21 , -19
      
      Y_train= y.iloc[:-23] #-21 , -19

      print(Y_train.tail(1))
      model.fit(X_train,Y_train)
      #now each day we will use the models to predict 
      flag=True
      for i in range(22,0,-1): #from 4/01 and later -->predictions
        if(flag):
          first=X.iloc[-(i+1):-i].index
          print('this is the first day of predictions', first)
          flag=False
        X_validation=X.iloc[-(i+1):-i] 
        # print(X_validation)
        Y_validation= y.iloc[-(i+1):-i]
        # print(Y_validation.index) 
        probability= model.predict_proba(X_validation)
        # print(probability[0])
        tmp.append(probability[0])
      X_validation=X.tail(1) 
      last=X_validation.index
      print('this is last for prediction',last)
      # print(X_validation)
      Y_validation= y.tail(1)
      print(Y_validation.index) 
      probability= model.predict_proba(X_validation)
      print(probability[0])
      tmp.append(probability[0])
      prob[j]=tmp
  return prob , first , last 

In [112]:
def RPS_data(prob_final,rd):
  RPS=pd.DataFrame()

  # RPS=RPS.drop('date',axis=1)
  # print(RPS.index[2])
  for k in prob_final.index: #for everyday of a stock->index
    prob=pd.DataFrame()
    prob1=pd.DataFrame()
    predic=pd.DataFrame()
    if(prob_final['date'][k]!=prob_final['date'].tail(1).values): #if it's not for the last day 
      #for historical data of two days 
      # prob=rd.loc[(rd.index.date==prob_final['date'][k])]
      # prob1=rd.loc[ (rd.index.date==prob_final['date'][k+1])]

      mask = (rd.loc[(rd.index.date>=prob_final['date'][k])].index)
      prob_table = rd.loc[mask]
      prob = prob_table.shift(-21).head(1)
      print('prob : ',prob)
      mask1 = (rd.loc[(rd.index.date>=prob_final['date'][k+1])].index)
      prob1_table = rd.loc[mask1]
      prob1 = prob1_table.shift(-21).head(1)
      print('prob1 : ',prob1)

      prob=create_historical(prob)
      prob1=create_historical(prob1)
      hist_data=prob.append(prob1)
      predic=prob_final.loc[prob_final['date']==prob_final['date'][k]] #find predictions of this day
      predic=predic.drop('date',axis=1)
      # print(predic)
      data=pd.DataFrame(index=predic.columns)
      data=data.reset_index()
      # print(data)
      data[['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=(0,0,0,0,0)
      for i in range(len(np.array_split(predic.values[0],46))-1):
        data.loc[i,['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=np.array_split(predic.values[0],46)[i][0]
      data.columns = ['ID','Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']
      # print(data)

      #now we have data->predictions and hist_data->historical data so now we can call the RPS
      rps=RPS_calculation(hist_data = hist_data , submission = data)['details']
      # print(rps) ->rps for every stock for one day 
      # print(rps['RPS'])
      tmp=pd.DataFrame(rps['RPS'].values).T
      # print(tmp.T)
      RPS=RPS.append(tmp,ignore_index=True)
      # print(RPS[RPS.index==k])

      
    else:
      #for the last day
      # prob=rd.loc[(rd.index.date==prob_final['date'][k])]
      # prob1=rd.loc[ (rd.index.date==rd.index.date[-1])]

      mask = (rd.loc[(rd.index.date>=prob_final['date'][k])].index)
      prob_table = rd.loc[mask]
      prob = prob_table.shift(-21).head(1)
      print('prob teleutaio :', prob)
      prob1 = pd.DataFrame(prob_table.iloc[1]).T
      print('prob1 : ',prob1)

      prob=create_historical(prob)
      prob1=create_historical(prob1)
      hist_data=prob.append(prob1)
      # print(hist_data)
      predic=prob_final.loc[prob_final['date']==prob_final['date'][k]] #find predictions of this day
      predic=predic.drop('date',axis=1)
      # print(predic)
      data=pd.DataFrame(index=predic.columns)
      data=data.reset_index()
      # print(data)
      data[['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=(0,0,0,0,0)
      for i in range(len(np.array_split(predic.values[0],46))-1):
        data.loc[i,['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=np.array_split(predic.values[0],46)[i][0]
      data.columns = ['ID','Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']
      # print(data)

      #now we have data->predictions and hist_data->historical data so now we can call the RPS
      rps=RPS_calculation(hist_data = hist_data , submission = data)['details']
      # print(rps) ->rps for every stock for one day 
      # print(rps['RPS'])
      tmp=pd.DataFrame(rps['RPS'].values).T
      # print(tmp.T)
      RPS=RPS.append(tmp,ignore_index=True)              
  return RPS

In [79]:
def create_RPS(prob_final,RPS):
  d=prob_final['date']
  name=prob_final.columns[:-1]
  # print(np.split(name.values,len(name)))
  # print(d.values)
  RPS['date']=d.values
  RPS=RPS.set_index('date')
  RPS.columns=assets
  return RPS

####**For 6 Mar.-1 Apr.** 

#####Features
***

In [80]:
stock=dict()
for i in assets:
  #παίρνω δεδομένα για 15 μήνες περιπου->από Ιανουάριο 2021 μέχρι Μάρτιο 2022
  stock[i]=pdr.get_data_yahoo(i,'2021-03-01','2022-04-02') 
  # print(df)
  #από αυτές κρατάμε μόνο τις πρώτες 5 στήλες 
  stock[i]=stock[i].iloc[:,:5]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [81]:
from datetime import datetime as dt 
for i in assets:
  stock[i].index= pd.to_datetime(stock[i].index)
print(len(stock['ABBV'].loc[(stock['ABBV'].index > '2022-03-04') & (stock['ABBV'].index < '2022-04-01')]))
# stock['ABBV']

19


In [82]:
for j in assets:
    df=stock[j] #all the days we need so we can fix the columns 
    # print('to df ftanei mexri ',stock[j].iloc[:-61].tail(1))
    # print(df)
    #τώρα θα προσθέσουμε τις καινούριες στήλες με τα features μας 
    df['(open2-open1)/open2']= (df['Open'].shift(2)-df['Open'].shift(1))/df['Open'].shift(2)
    df['(high2-high1)/high2']= (df['High'].shift(2)-df['High'].shift(1))/df['High'].shift(2)
    df['(low2-low1)/low2']= (df['Low'].shift(2)-df['Low'].shift(1))/df['Low'].shift(2)
    df['(Adj Close2-Adj Close1)/Adj Close2']=(df['Adj Close'].shift(2)-df['Adj Close'].shift(1))/df['Adj Close'].shift(2)
    df['av_3']=df['Adj Close'].shift(1).rolling(window=3).mean().pct_change()
    df['av_6']=df['Adj Close'].shift(1).rolling(window=6).mean().pct_change()
    df['av_10']=df['Adj Close'].shift(1).rolling(window=10).mean().pct_change()
    df['av_14']=df['Adj Close'].shift(1).rolling(window=14).mean().pct_change()
    # df['av_14']=df['Adj Close'].shift(1).rolling(window=14).mean().pct_change()
    df['high1-low1/high1']=(df['High'].shift(1)-df['Low'].shift(1))/df['High'].shift(1)
    df['close1-open1/close1']=(df['Close'].shift(1)-df['Open'].shift(1))/df['Close'].shift(1)
    df['RSI']= ta.RSI(np.array(df['Adj Close'].shift(1)),timeperiod=6)/100
    df['ADX'] = ta.ADX(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1), timeperiod=6)/100
    df['RSI_14']= ta.RSI(np.array(df['Adj Close'].shift(1)),timeperiod=14)/100
    df['ADX_14'] = ta.ADX(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1), timeperiod=14)/100
    df['TRIX_14'] =ta.TRIX(df['Adj Close'].shift(1),timeperiod=14)
    df['TRIX_10'] =ta.TRIX(df['Adj Close'].shift(1),timeperiod=10)
    df['WILLR']= ta.WILLR(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1),timeperiod=14)/100
    df['std_dev']=df['Adj Close'].rolling(window=10).std().shift(1)
    df['dist_from_mean_10'] = (df['Adj Close'].shift(1) - (df['Adj Close'].shift(1).rolling(window=10).mean()))
    df['dist_from_mean_6'] = df['Adj Close'].shift(1) - (df['Adj Close'].shift(1).rolling(window=6).mean())
    df['MACD']= ta.MACD(df['Adj Close'].shift(1),fastperiod=5,slowperiod=14, signalperiod=8)[0]
    df['PPO']= ta.PPO(df['Adj Close'].shift(1), fastperiod= 5,slowperiod=14,matype=0)
    df['CMO_14']= ta.CMO(df['Adj Close'].shift(1),timeperiod=14)
    df = df.iloc[1: , :]
    df['absolute rank']= jk_feature[j].astype(int).shift(1)
    df['Rank']= rd3[j]
    df1=df.dropna()
    stock[j]=df1.copy()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [85]:
for i in assets:
  print(stock[i].loc[stock[i].index=='2022-03-02']['Rank'])

Date
2022-03-02    3
Name: Rank, dtype: int64
Date
2022-03-02    5
Name: Rank, dtype: int64
Date
2022-03-02    5
Name: Rank, dtype: int64
Date
2022-03-02    4
Name: Rank, dtype: int64
Date
2022-03-02    1
Name: Rank, dtype: int64
Date
2022-03-02    1
Name: Rank, dtype: int64
Date
2022-03-02    2
Name: Rank, dtype: int64
Date
2022-03-02    5
Name: Rank, dtype: int64
Date
2022-03-02    3
Name: Rank, dtype: int64
Date
2022-03-02    2
Name: Rank, dtype: int64
Date
2022-03-02    4
Name: Rank, dtype: int64
Date
2022-03-02    4
Name: Rank, dtype: int64
Date
2022-03-02    4
Name: Rank, dtype: int64
Date
2022-03-02    3
Name: Rank, dtype: int64
Date
2022-03-02    3
Name: Rank, dtype: int64
Date
2022-03-02    1
Name: Rank, dtype: int64
Date
2022-03-02    5
Name: Rank, dtype: int64
Date
2022-03-02    4
Name: Rank, dtype: int64
Date
2022-03-02    5
Name: Rank, dtype: int64
Date
2022-03-02    3
Name: Rank, dtype: int64
Date
2022-03-02    2
Name: Rank, dtype: int64
Date
2022-03-02    1
Name: Rank, d

#####Logistic Regression (baseline)
***

In [83]:
from sklearn.linear_model import LogisticRegression
baseline= LogisticRegression()
#---predictions---#
prob1,first,last=predictions(baseline,assets,stock,29)
#---probabilities---#
mask = ((stock['ABBV'].index.values >= first.values) & (stock['ABBV'].index.values <= last.values))
prob1['date']=stock['ABBV'].loc[mask].index
prob1_final=pd.DataFrame.from_dict(prob1)
prob1_final.set_index('date')
RPS1= RPS_data(prob1_final,rd)
RPS1= create_RPS(prob1_final,RPS1)
RPS1
# print(RPS1.mean(axis=1).mean())

Date
2022-03-01    4
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-02'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-04-01'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-04-01'], dtype='datetime64[ns]', name='Date', freq=None)
[0.09613247 0.1995014  0.25245709 0.19440206 0.25750698]
Date
2022-03-01    5
Name: Rank, dtype: int64


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


this is the first day of predictions DatetimeIndex(['2022-03-02'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-04-01'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-04-01'], dtype='datetime64[ns]', name='Date', freq=None)
[0.215258   0.16377135 0.07594947 0.38977479 0.1552464 ]
Date
2022-03-01    5
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-02'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-04-01'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-04-01'], dtype='datetime64[ns]', name='Date', freq=None)
[0.0858793  0.23605731 0.35078569 0.27260623 0.05467146]
Date
2022-03-01    1
Name: Rank, dtype: int64


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


this is the first day of predictions DatetimeIndex(['2022-03-02'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-04-01'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-04-01'], dtype='datetime64[ns]', name='Date', freq=None)
[0.18873533 0.05998845 0.16487211 0.31479972 0.27160439]
Date
2022-03-01    1
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-02'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-04-01'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-04-01'], dtype='datetime64[ns]', name='Date', freq=None)
[0.21476221 0.08778371 0.16081503 0.15191643 0.38472263]
Date
2022-03-01    1
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-02'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-04-01'], dtype='datetime64[ns]',

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV        AEP         AIZ      ALLE        AMAT  \
Date                                                                  
2022-03-31  146.499664  96.603081  179.417999  114.3153  112.406975   

                   AMP   AMZN         AVB         AVY         AXP  ...  \
Date                                                               ...   
2022-03-31  266.332184  124.5  219.365616  177.753952  170.980133  ...   

                   PRU       PYPL          RE       ROL        ROST  \
Date                                                                  
2022-03-31  105.849098  91.529999  274.105255  33.01482  100.586754   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-03-31  497.807159  307.029999  210.753494  201.466431  84.710785  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ        ALLE        AMAT  \
Date                

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob teleutaio :                   ABBV        AEP         AIZ        ALLE        AMAT  \
Date                                                                    
2022-04-01  148.213638  97.046082  181.640381  114.932793  113.232841   

                  AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                   ...   
2022-04-01  269.58197  124.253502  220.645218  179.657928  171.468307  ...   

                   PRU       PYPL          RE       ROL       ROST  \
Date                                                                 
2022-04-01  107.137924  90.709999  276.180115  32.52755  99.723053   

                   UNH         URI           V        VRSK       XOM  
Date                                                                  
2022-04-01  496.287384  312.450012  207.794388  199.403824  86.45578  

[1 rows x 45 columns]
prob1 :                    ABBV       AEP         AIZ        ALLE        AMAT  \


,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-03-02,0.067308,0.156415,0.263564,0.168373,0.267549,0.259171,0.083278,0.250050,0.072807,0.116244,...,0.067694,0.173330,0.199985,0.186155,0.077491,0.079806,0.158946,0.191521,0.129681,0.100919
2022-03-03,0.194482,0.245378,0.264988,0.061820,0.138274,0.028376,0.287692,0.253216,0.082045,0.090941,...,0.254314,0.378277,0.259105,0.205234,0.366254,0.155343,0.328131,0.106730,0.103561,0.109388
2022-03-04,0.214251,0.065768,0.059407,0.080635,0.200757,0.045620,0.212017,0.056695,0.145905,0.176155,...,0.122553,0.073537,0.135606,0.091689,0.130673,0.195415,0.229930,0.080710,0.196108,0.114360
2022-03-07,0.193747,0.129851,0.130768,0.203962,0.144997,0.310551,0.212978,0.296143,0.094101,0.132603,...,0.108070,0.081995,0.125870,0.145282,0.150629,0.141412,0.173838,0.304300,0.069510,0.141622
2022-03-08,0.244335,0.390892,0.047732,0.115406,0.251863,0.305077,0.131069,0.231205,0.093995,0.155588,...,0.193253,0.181369,0.141157,0.470150,0.108748,0.096279,0.154080,0.403199,0.070411,0.167912
2022-03-09,0.089040,0.282442,0.046034,0.283771,0.172475,0.170650,0.256618,0.071661,0.152912,0.069250,...,0.245924,0.129922,0.016528,0.078614,0.107505,0.275578,0.172772,0.124337,0.221176,0.188129
2022-03-10,0.318917,0.128311,0.265770,0.288303,0.172585,0.130121,0.116707,0.067125,0.330081,0.438567,...,0.121948,0.116255,0.184974,0.328440,0.347087,0.052417,0.188151,0.091533,0.104513,0.132013
2022-03-11,0.291238,0.235110,0.057281,0.078268,0.217006,0.179449,0.165131,0.107472,0.088233,0.249192,...,0.125210,0.156194,0.296130,0.085813,0.367607,0.069166,0.181842,0.235763,0.200534,0.359410
2022-03-14,0.264303,0.295301,0.106876,0.218343,0.203207,0.280316,0.327029,0.136570,0.229239,0.142587,...,0.057212,0.143784,0.036330,0.137467,0.117135,0.071318,0.085739,0.125163,0.361735,0.268755


In [86]:
print(RPS1.mean(axis=1).mean())

0.16882710594203165


In [87]:
print(RPS1.mean(axis=1))

date
2022-03-02    0.160921
2022-03-03    0.190660
2022-03-04    0.143261
2022-03-07    0.139277
2022-03-08    0.178488
2022-03-09    0.160906
2022-03-10    0.183683
2022-03-11    0.169292
2022-03-14    0.169471
2022-03-15    0.152380
2022-03-16    0.178339
2022-03-17    0.184488
2022-03-18    0.164355
2022-03-21    0.178816
2022-03-22    0.181526
2022-03-23    0.189868
2022-03-24    0.169597
2022-03-25    0.139205
2022-03-28    0.165471
2022-03-29    0.171984
2022-03-30    0.177390
2022-03-31    0.167375
2022-04-01    0.166270
dtype: float64


In [88]:
#save results 
RPS1.to_excel('6_Mar_1_Apr-log_reg(baseline)-real_data.xlsx')
prob1_final.set_index('date',inplace=True)
prob1_final.to_excel('6_Mar_1_Apr-PROBABILITIES-log_reg(baseline)-real_data.xlsx')

#####After feature importances 
***

In [89]:
stock1=stock.copy()
# print(stock2)
for j in assets:
  stock1[j]=stock1[j].drop(['absolute rank','CMO_14','RSI','RSI_14','PPO','MACD','TRIX_14','TRIX_10','dist_from_mean_6','WILLR','dist_from_mean_10','std_dev','(Adj Close2-Adj Close1)/Adj Close2','ADX_14'],axis=1)

In [90]:
from sklearn.linear_model import LogisticRegression
model1= LogisticRegression(max_iter=500)
#---predictions---#
prob2 ,first,last =predictions(model1,assets,stock1,15)
#---probabilities---#
mask = ((stock['ABBV'].index.values >= first.values) & (stock['ABBV'].index.values <= last.values))
prob2['date']=stock['ABBV'].loc[mask].index
prob2_final=pd.DataFrame.from_dict(prob2)
prob2_final.set_index('date')
RPS2= RPS_data(prob2_final,rd)
RPS2= create_RPS(prob2_final,RPS2)
RPS2
# print(RPS2.mean(axis=1).mean())

Date
2022-03-01    4
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-02'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-04-01'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-04-01'], dtype='datetime64[ns]', name='Date', freq=None)
[0.12914029 0.18811237 0.20296961 0.20130315 0.27847459]
Date
2022-03-01    5
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-02'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-04-01'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-04-01'], dtype='datetime64[ns]', name='Date', freq=None)
[0.17393576 0.21097747 0.19842393 0.22778397 0.18887888]
Date
2022-03-01    5
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-02'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction Datetim

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kerne

prob :                    ABBV        AEP        AIZ        ALLE        AMAT  \
Date                                                                   
2022-03-03  159.814148  99.930489  183.96199  111.068497  126.775017   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-03-03  294.668579  163.559998  252.387024  173.579086  185.944336  ...   

                   PRU        PYPL          RE        ROL       ROST  \
Date                                                                   
2022-03-03  115.603111  116.669998  303.419739  34.615841  89.954269   

                   UNH    URI           V        VRSK        XOM  
Date                                                              
2022-03-03  509.170868  354.0  225.529037  213.353729  81.485481  

[1 rows x 45 columns]
prob1 :                    ABBV       AEP         AIZ        ALLE        AMAT  \
Date           

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV       AEP         AIZ        ALLE        AMAT  \
Date                                                                   
2022-03-04  159.038071  99.08387  180.370453  110.849396  128.794907   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-03-04  294.787476  168.346497  248.984695  173.113022  186.431122  ...   

                   PRU        PYPL          RE        ROL       ROST  \
Date                                                                   
2022-03-04  113.630821  121.860001  293.727295  34.894279  92.495743   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-03-04  506.618011  349.899994  226.903992  216.233383  81.524696  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ        ALLE        AMAT  \

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV        AEP         AIZ      ALLE        AMAT  \
Date                                                                  
2022-03-31  146.499664  96.603081  179.417999  114.3153  112.406975   

                   AMP   AMZN         AVB         AVY         AXP  ...  \
Date                                                               ...   
2022-03-31  266.332184  124.5  219.365616  177.753952  170.980133  ...   

                   PRU       PYPL          RE       ROL        ROST  \
Date                                                                  
2022-03-31  105.849098  91.529999  274.105255  33.01482  100.586754   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-03-31  497.807159  307.029999  210.753494  201.466431  84.710785  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ        ALLE        AMAT  \
Date                

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob teleutaio :                   ABBV        AEP         AIZ        ALLE        AMAT  \
Date                                                                    
2022-04-01  148.213638  97.046082  181.640381  114.932793  113.232841   

                  AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                   ...   
2022-04-01  269.58197  124.253502  220.645218  179.657928  171.468307  ...   

                   PRU       PYPL          RE       ROL       ROST  \
Date                                                                 
2022-04-01  107.137924  90.709999  276.180115  32.52755  99.723053   

                   UNH         URI           V        VRSK       XOM  
Date                                                                  
2022-04-01  496.287384  312.450012  207.794388  199.403824  86.45578  

[1 rows x 45 columns]
prob1 :                    ABBV       AEP         AIZ        ALLE        AMAT  \


,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-03-02,0.073087,0.236539,0.258893,0.130706,0.231048,0.274962,0.114822,0.236819,0.066922,0.125299,...,0.123646,0.131386,0.211808,0.258020,0.108991,0.073032,0.128237,0.237154,0.130546,0.097727
2022-03-03,0.140606,0.123301,0.274791,0.074427,0.131275,0.074467,0.247429,0.258392,0.101405,0.119512,...,0.239455,0.237174,0.268353,0.110666,0.267406,0.125689,0.208735,0.112098,0.102312,0.098695
2022-03-04,0.206152,0.113670,0.057925,0.105284,0.231647,0.074158,0.210527,0.072957,0.130731,0.127228,...,0.122574,0.181293,0.113912,0.086465,0.094293,0.230877,0.210804,0.072974,0.241573,0.098945
2022-03-07,0.206049,0.237446,0.111342,0.226380,0.143877,0.269177,0.207316,0.241487,0.101585,0.126544,...,0.079005,0.186451,0.127109,0.086362,0.135876,0.228608,0.210640,0.261575,0.064141,0.130083
2022-03-08,0.205498,0.252343,0.058426,0.124996,0.200746,0.265514,0.107105,0.260612,0.101969,0.127803,...,0.233262,0.131816,0.124248,0.217978,0.135167,0.105569,0.213065,0.264534,0.065525,0.217903
2022-03-09,0.095870,0.123478,0.058501,0.227900,0.236110,0.108358,0.202670,0.072885,0.130146,0.073060,...,0.241088,0.188073,0.076007,0.135713,0.137811,0.229277,0.208625,0.073488,0.133616,0.225094
2022-03-10,0.291904,0.072167,0.261666,0.272763,0.235512,0.106574,0.103169,0.072862,0.286803,0.263712,...,0.123006,0.130605,0.120374,0.218178,0.263748,0.072961,0.139815,0.073532,0.068758,0.215329
2022-03-11,0.291162,0.238577,0.058446,0.074297,0.238238,0.131417,0.144145,0.108191,0.101364,0.130774,...,0.122943,0.181895,0.255883,0.084504,0.262369,0.073123,0.231406,0.264023,0.134772,0.222155
2022-03-14,0.291036,0.252378,0.111849,0.275766,0.200931,0.227439,0.253731,0.121952,0.227819,0.222244,...,0.079159,0.181394,0.076239,0.136963,0.094012,0.072918,0.227710,0.109244,0.268501,0.134398


In [91]:
print(RPS2.mean(axis=1).mean())

0.1588008486680134


In [92]:
print(RPS2.mean(axis=1).head(1))

date
2022-03-02    0.160608
dtype: float64


In [93]:

#save results 
RPS2.to_excel('6_Mar_1_Apr-log_reg(with_feature_imp)-real_data.xlsx')
prob2_final.set_index('date',inplace=True)
prob2_final.to_excel('6_Mar_1_Apr-PROBABILITIES-log_reg(with_feature_imp)-real_data.xlsx')

####**For 3 Apr.-29 Apr.**

#####Features
***

In [94]:
stock=dict()
for i in assets:
  #παίρνω δεδομένα για 15 μήνες περιπου->από Ιανουάριο 2021 μέχρι Μάρτιο 2022
  stock[i]=pdr.get_data_yahoo(i,'2021-04-01','2022-05-02') 
  # print(df)
  #από αυτές κρατάμε μόνο τις πρώτες 5 στήλες 
  stock[i]=stock[i].iloc[:,:5]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [95]:
stock

{'ABBV':                   Open        High         Low       Close   Adj Close
 Date                                                                  
 2021-04-01  108.919998  108.980003  107.320000  108.519997  101.861618
 2021-04-05  109.080002  109.080002  105.400002  106.139999   99.627640
 2021-04-06  105.849998  106.279999  104.879997  105.379997   98.914276
 2021-04-07  105.529999  106.129997  104.169998  105.209999   98.754707
 2021-04-08  105.400002  106.919998  105.349998  106.099998   99.590096
 ...                ...         ...         ...         ...         ...
 2022-04-25  154.839996  156.639999  152.839996  156.300003  154.851547
 2022-04-26  156.440002  159.690002  155.820007  156.179993  154.732651
 2022-04-27  156.559998  159.500000  154.559998  157.619995  156.159302
 2022-04-28  158.360001  158.440002  154.339996  156.309998  154.861450
 2022-04-29  149.559998  149.860001  139.929993  146.880005  145.518845
 
 [273 rows x 5 columns],
 'AEP':                   Ope

In [96]:
from datetime import datetime as dt 
for i in assets:
  stock[i].index= pd.to_datetime(stock[i].index)
print(len(stock['ABBV'].loc[(stock['ABBV'].index > '2022-04-01') & (stock['ABBV'].index <= '2022-04-29')]))

19


In [97]:
for j in assets:
    df=stock[j] #all the days we need so we can fix the columns 
    # print('to df ftanei mexri ',stock[j].iloc[:-61].tail(1))
    # print(df)
    #τώρα θα προσθέσουμε τις καινούριες στήλες με τα features μας 
    df['(open2-open1)/open2']= (df['Open'].shift(1)-df['Open'])/df['Open'].shift(1)
    df['(high2-high1)/high2']= (df['High'].shift(1)-df['High'])/df['High'].shift(1)
    df['(low2-low1)/low2']= (df['Low'].shift(1)-df['Low'])/df['Low'].shift(1)
    df['(Adj Close2-Adj Close1)/Adj Close2']=(df['Adj Close'].shift(1)-df['Adj Close'])/df['Adj Close'].shift(1)
    df['av_3']=df['Adj Close'].rolling(window=3).mean().pct_change()
    df['av_6']=df['Adj Close'].rolling(window=6).mean().pct_change()
    df['av_10']=df['Adj Close'].rolling(window=10).mean().pct_change()
    df['av_14']=df['Adj Close'].rolling(window=14).mean().pct_change()
    # df['av_14']=df['Adj Close'].rolling(window=14).mean().pct_change()
    df['high1-low1/high1']=(df['High']-df['Low'])/df['High']
    df['close1-open1/close1']=(df['Close']-df['Open'])/df['Close']
    df['RSI']= ta.RSI(np.array(df['Adj Close']),timeperiod=6)/100
    df['ADX'] = ta.ADX(df['High'], df['Low'], df['Close'], timeperiod=6)/100
    df['RSI_14']= ta.RSI(np.array(df['Adj Close']),timeperiod=14)/100
    df['ADX_14'] = ta.ADX(df['High'], df['Low'], df['Close'], timeperiod=14)/100
    df['TRIX_14'] =ta.TRIX(df['Adj Close'],timeperiod=14)
    df['TRIX_10'] =ta.TRIX(df['Adj Close'],timeperiod=10)
    df['WILLR']= ta.WILLR(df['High'], df['Low'], df['Close'],timeperiod=14)/100
    df['std_dev']=df['Adj Close'].rolling(window=10).std()
    df['dist_from_mean_10'] = (df['Adj Close'] - (df['Adj Close'].rolling(window=10).mean()))
    df['dist_from_mean_6'] = df['Adj Close'] - (df['Adj Close'].rolling(window=6).mean())
    df['MACD']= ta.MACD(df['Adj Close'],fastperiod=5,slowperiod=14, signalperiod=8)[0]
    df['PPO']= ta.PPO(df['Adj Close'], fastperiod= 5,slowperiod=14,matype=0)
    df['CMO_14']= ta.CMO(df['Adj Close'],timeperiod=14)
    df = df.iloc[1: , :]
    df['absolute rank']= jk_feature[j].astype(int).shift(1)
    df['Rank']= rd3[j]
    df1=df.dropna()
    stock[j]=df1.copy()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [100]:
#for target in 4 april I give the rank of 
for i in assets:
  print(stock[i].loc[stock[i].index=='2022-03-29']['Rank'])

Date
2022-03-29    1
Name: Rank, dtype: int64
Date
2022-03-29    2
Name: Rank, dtype: int64
Date
2022-03-29    4
Name: Rank, dtype: int64
Date
2022-03-29    1
Name: Rank, dtype: int64
Date
2022-03-29    2
Name: Rank, dtype: int64
Date
2022-03-29    4
Name: Rank, dtype: int64
Date
2022-03-29    1
Name: Rank, dtype: int64
Date
2022-03-29    1
Name: Rank, dtype: int64
Date
2022-03-29    5
Name: Rank, dtype: int64
Date
2022-03-29    2
Name: Rank, dtype: int64
Date
2022-03-29    4
Name: Rank, dtype: int64
Date
2022-03-29    5
Name: Rank, dtype: int64
Date
2022-03-29    4
Name: Rank, dtype: int64
Date
2022-03-29    2
Name: Rank, dtype: int64
Date
2022-03-29    3
Name: Rank, dtype: int64
Date
2022-03-29    3
Name: Rank, dtype: int64
Date
2022-03-29    1
Name: Rank, dtype: int64
Date
2022-03-29    5
Name: Rank, dtype: int64
Date
2022-03-29    4
Name: Rank, dtype: int64
Date
2022-03-29    5
Name: Rank, dtype: int64
Date
2022-03-29    2
Name: Rank, dtype: int64
Date
2022-03-29    5
Name: Rank, d

#####Logistic Regression (baseline)
***

In [101]:
from sklearn.linear_model import LogisticRegression
baseline= LogisticRegression()
#---predictions---#
prob1,first,last=predictions(baseline,assets,stock,29)
#---probabilities---#
mask = ((stock['ABBV'].index.values >= first.values) & (stock['ABBV'].index.values <= last.values))
prob1['date']=stock['ABBV'].loc[mask].index
prob1_final=pd.DataFrame.from_dict(prob1)
prob1_final.set_index('date')
RPS1= RPS_data(prob1_final,rd)
RPS1= create_RPS(prob1_final,RPS1)
RPS1
# print(RPS1.mean(axis=1).mean())

Date
2022-03-28    1
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-29'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-04-29'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-04-29'], dtype='datetime64[ns]', name='Date', freq=None)
[0.03077792 0.0447171  0.36769488 0.53846622 0.01834388]
Date
2022-03-28    4
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-29'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-04-29'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-04-29'], dtype='datetime64[ns]', name='Date', freq=None)
[0.33549641 0.13517566 0.12424852 0.24076124 0.16431816]
Date
2022-03-28    2
Name: Rank, dtype: int64


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


this is the first day of predictions DatetimeIndex(['2022-03-29'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-04-29'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-04-29'], dtype='datetime64[ns]', name='Date', freq=None)
[0.05880907 0.11434664 0.40481641 0.2845259  0.13750198]
Date
2022-03-28    4
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-29'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-04-29'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-04-29'], dtype='datetime64[ns]', name='Date', freq=None)
[0.27729795 0.19990655 0.22079021 0.17669833 0.12530696]
Date
2022-03-28    5
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-29'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-04-29'], dtype='datetime64[ns]',

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


this is the first day of predictions DatetimeIndex(['2022-03-29'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-04-29'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-04-29'], dtype='datetime64[ns]', name='Date', freq=None)
[0.34541335 0.09404276 0.21683202 0.17897186 0.16474   ]
Date
2022-03-28    5
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-29'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-04-29'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-04-29'], dtype='datetime64[ns]', name='Date', freq=None)
[0.27819691 0.34644648 0.02862367 0.08473299 0.26199995]
Date
2022-03-28    1
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-29'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-04-29'], dtype='datetime64[ns]',

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV         AEP         AIZ        ALLE       AMAT  \
Date                                                                    
2022-04-28  148.609924  102.762794  180.665848  113.080322  119.17524   

                   AMP      AMZN         AVB         AVY         AXP  ...  \
Date                                                                  ...   
2022-04-28  277.202972  115.1465  209.208252  172.161026  168.967667  ...   

                  PRU       PYPL          RE        ROL       ROST  \
Date                                                                 
2022-04-28  105.03669  85.209999  283.079163  36.513096  86.678146   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-04-28  503.727386  297.459991  212.503616  178.010666  96.650909  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP        AIZ        ALLE        AMAT  \
Date    

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob teleutaio :                   ABBV         AEP        AIZ        ALLE        AMAT  \
Date                                                                    
2022-04-29  146.004288  101.234802  175.96521  111.197975  116.990822   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-04-29  274.993225  120.209503  206.282059  171.862244  168.190582  ...   

                   PRU       PYPL          RE        ROL       ROST  \
Date                                                                  
2022-04-29  105.016922  85.209999  280.802887  35.366138  84.404716   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-04-29  493.466309  298.179993  211.794876  174.294022  95.076218  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ      ALLE        

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-03-29,0.386590,0.164849,0.108583,0.186911,0.142075,0.100777,0.233957,0.251223,0.271371,0.142328,...,0.082907,0.187274,0.171986,0.088620,0.208784,0.079180,0.264109,0.122692,0.190119,0.090053
2022-03-30,0.089465,0.183787,0.030800,0.102438,0.202342,0.091747,0.098561,0.195234,0.131079,0.335596,...,0.082510,0.298709,0.143612,0.155324,0.259513,0.354199,0.230241,0.114391,0.187820,0.099793
2022-03-31,0.090460,0.107149,0.108801,0.107136,0.114502,0.097067,0.092464,0.058025,0.149669,0.140968,...,0.105355,0.152971,0.126277,0.225719,0.261482,0.180568,0.253907,0.148208,0.366097,0.232396
2022-04-01,0.335437,0.152843,0.125987,0.091486,0.166922,0.201654,0.179570,0.254451,0.057006,0.115955,...,0.134786,0.103179,0.139989,0.263175,0.111849,0.340293,0.061487,0.146954,0.342071,0.243180
2022-04-04,0.145225,0.128202,0.211730,0.101942,0.239017,0.278525,0.176238,0.149547,0.056967,0.109987,...,0.081949,0.179332,0.229153,0.131020,0.121374,0.075980,0.229322,0.228116,0.047976,0.141961
2022-04-05,0.071047,0.071575,0.050728,0.119691,0.109494,0.214394,0.228063,0.286599,0.093752,0.238949,...,0.128208,0.154629,0.097354,0.259552,0.200013,0.189156,0.249175,0.052732,0.196890,0.243940
2022-04-06,0.141372,0.147352,0.157071,0.230000,0.218881,0.175827,0.134137,0.094372,0.137258,0.118519,...,0.100875,0.120722,0.090338,0.122086,0.095246,0.074015,0.151092,0.175720,0.196554,0.238897
2022-04-07,0.245516,0.132793,0.090511,0.274223,0.252368,0.142059,0.110714,0.150439,0.253256,0.107470,...,0.130634,0.102978,0.098485,0.274716,0.220560,0.065961,0.225124,0.102633,0.076295,0.119270
2022-04-08,0.278021,0.160984,0.060671,0.110483,0.180955,0.106353,0.141354,0.228592,0.115371,0.133226,...,0.093937,0.146142,0.081048,0.185472,0.106268,0.127514,0.237106,0.251985,0.161446,0.158961


In [102]:
print(RPS1.mean(axis=1).mean())

0.16466499634735024


In [103]:
print(RPS1.mean(axis=1).head(1))

date
2022-03-29    0.163245
dtype: float64


In [104]:
#save results 
RPS1.to_excel('4_Apr_29_Apr-log_reg(baseline)-real_data.xlsx')
prob1_final.set_index('date',inplace=True)
prob1_final.to_excel('4_Apr_29_Apr-PROBABILITIES-log_reg(baseline)-real_data.xlsx')

#####After feature importances
***

In [105]:
stock1=stock.copy()
# print(stock2)
for j in assets:
  stock1[j]=stock1[j].drop(['absolute rank','CMO_14','RSI','RSI_14','PPO','MACD','TRIX_14','TRIX_10','dist_from_mean_6','WILLR','dist_from_mean_10','std_dev','(Adj Close2-Adj Close1)/Adj Close2','ADX_14'],axis=1)

In [106]:
from sklearn.linear_model import LogisticRegression
model1= LogisticRegression(max_iter=500)
#---predictions---#
prob2 ,first,last =predictions(model1,assets,stock1,15)
#---probabilities---#
mask = ((stock['ABBV'].index.values >= first.values) & (stock['ABBV'].index.values <= last.values))
prob2['date']=stock['ABBV'].loc[mask].index
prob2_final=pd.DataFrame.from_dict(prob2)
prob2_final.set_index('date')
RPS2= RPS_data(prob2_final,rd)
RPS2= create_RPS(prob2_final,RPS2)
RPS2
# print(RPS2.mean(axis=1).mean())

Date
2022-03-28    1
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-29'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-04-29'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-04-29'], dtype='datetime64[ns]', name='Date', freq=None)
[0.14366412 0.17569085 0.20039132 0.23617478 0.24407893]
Date
2022-03-28    4
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-29'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-04-29'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-04-29'], dtype='datetime64[ns]', name='Date', freq=None)
[0.16748712 0.2062008  0.18288511 0.24439445 0.19903252]
Date
2022-03-28    2
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-29'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction Datetim

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kerne

prob :                    ABBV        AEP         AIZ        ALLE        AMAT  \
Date                                                                    
2022-03-30  145.518845  97.567825  180.449829  113.777489  109.800034   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-03-30  263.042816  124.281502  225.644547  179.092697  174.058624  ...   

                   PRU   PYPL          RE        ROL       ROST         UNH  \
Date                                                                          
2022-03-30  105.946739  87.93  271.417877  33.352924  99.047974  505.157806   

                   URI          V        VRSK        XOM  
Date                                                      
2022-03-30  316.519989  212.34761  203.319763  83.573593  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ      ALLE        AMAT  \
Date            

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV        AEP         AIZ      ALLE        AMAT  \
Date                                                                  
2022-03-31  146.499664  96.603081  179.417999  114.3153  112.406975   

                   AMP   AMZN         AVB         AVY         AXP  ...  \
Date                                                               ...   
2022-03-31  266.332184  124.5  219.365616  177.753952  170.980133  ...   

                   PRU       PYPL          RE       ROL        ROST  \
Date                                                                  
2022-03-31  105.849098  91.529999  274.105255  33.01482  100.586754   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-03-31  497.807159  307.029999  210.753494  201.466431  84.710785  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ        ALLE        AMAT  \
Date                

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV         AEP         AIZ        ALLE       AMAT  \
Date                                                                    
2022-04-28  148.609924  102.762794  180.665848  113.080322  119.17524   

                   AMP      AMZN         AVB         AVY         AXP  ...  \
Date                                                                  ...   
2022-04-28  277.202972  115.1465  209.208252  172.161026  168.967667  ...   

                  PRU       PYPL          RE        ROL       ROST  \
Date                                                                 
2022-04-28  105.03669  85.209999  283.079163  36.513096  86.678146   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-04-28  503.727386  297.459991  212.503616  178.010666  96.650909  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP        AIZ        ALLE        AMAT  \
Date    

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob teleutaio :                   ABBV         AEP        AIZ        ALLE        AMAT  \
Date                                                                    
2022-04-29  146.004288  101.234802  175.96521  111.197975  116.990822   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-04-29  274.993225  120.209503  206.282059  171.862244  168.190582  ...   

                   PRU       PYPL          RE        ROL       ROST  \
Date                                                                  
2022-04-29  105.016922  85.209999  280.802887  35.366138  84.404716   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-04-29  493.466309  298.179993  211.794876  174.294022  95.076218  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ      ALLE        

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-03-29,0.301338,0.131694,0.104921,0.229114,0.139619,0.111855,0.191942,0.262566,0.275430,0.120540,...,0.076833,0.152327,0.075128,0.118860,0.220452,0.071426,0.228935,0.120604,0.130316,0.122726
2022-03-30,0.090126,0.131536,0.057270,0.122020,0.212328,0.111479,0.093753,0.265889,0.102157,0.247929,...,0.077052,0.281651,0.120334,0.127460,0.247639,0.308071,0.216337,0.120063,0.129758,0.121075
2022-03-31,0.090166,0.079963,0.104897,0.077034,0.115023,0.106220,0.105442,0.075592,0.130350,0.118015,...,0.121750,0.148950,0.074347,0.240042,0.223718,0.150705,0.227289,0.236128,0.277517,0.201665
2022-04-01,0.300371,0.131236,0.125324,0.077384,0.209769,0.224817,0.203469,0.267063,0.065781,0.126078,...,0.120285,0.119909,0.074057,0.224095,0.125623,0.298849,0.107890,0.120992,0.276433,0.200304
2022-04-04,0.186869,0.106371,0.246776,0.114120,0.223379,0.268142,0.205315,0.101521,0.065914,0.117069,...,0.076913,0.153428,0.229930,0.121615,0.118733,0.098173,0.213342,0.237521,0.053458,0.119555
2022-04-05,0.090684,0.107214,0.057321,0.113854,0.114722,0.135893,0.204752,0.268224,0.104473,0.243381,...,0.121707,0.153614,0.223349,0.223372,0.222569,0.212675,0.212922,0.120016,0.130112,0.199838
2022-04-06,0.090603,0.227540,0.124376,0.259624,0.230517,0.074814,0.207281,0.128531,0.128975,0.116264,...,0.118565,0.128434,0.076203,0.121642,0.117314,0.069758,0.128151,0.234078,0.129502,0.252688
2022-04-07,0.090332,0.130210,0.104871,0.224495,0.209611,0.074460,0.095392,0.260791,0.234910,0.126350,...,0.122134,0.119101,0.076629,0.241759,0.218931,0.070399,0.226145,0.077569,0.106292,0.121335
2022-04-08,0.090471,0.229448,0.057365,0.110734,0.224591,0.073895,0.202683,0.238073,0.128958,0.117149,...,0.076643,0.162427,0.114252,0.223289,0.112702,0.091868,0.216474,0.248168,0.129478,0.204838


In [107]:
print(RPS2.mean(axis=1).mean())

0.15918967452154698


In [108]:
print(RPS2.mean(axis=1))

date
2022-03-29    0.160179
2022-03-30    0.166981
2022-03-31    0.154757
2022-04-01    0.163047
2022-04-04    0.150418
2022-04-05    0.152142
2022-04-06    0.158872
2022-04-07    0.157943
2022-04-08    0.145455
2022-04-11    0.165834
2022-04-12    0.163040
2022-04-13    0.150394
2022-04-14    0.162428
2022-04-18    0.158878
2022-04-19    0.168557
2022-04-20    0.164445
2022-04-21    0.153657
2022-04-22    0.151739
2022-04-25    0.162710
2022-04-26    0.162121
2022-04-27    0.162844
2022-04-28    0.164501
2022-04-29    0.160421
dtype: float64


In [110]:
#save results 
RPS2.to_excel('4_Apr_29_Apr-log_reg(with_feature_imp)-real_data.xlsx')
prob2_final.set_index('date',inplace=True)
prob2_final.to_excel('4_Apr_29_Apr-PROBABILITIES-log_reg(with_feature_imp)-real_data.xlsx')

KeyError: ignored

####**For 1 May-27 May**

#####Features
***

In [130]:
stock=dict()
for i in assets:
  #παίρνω δεδομένα για 15 μήνες περιπου->από Ιανουάριο 2021 μέχρι Μάρτιο 2022
  stock[i]=pdr.get_data_yahoo(i,'2021-05-01','2022-05-28') 
  # print(df)
  #από αυτές κρατάμε μόνο τις πρώτες 5 στήλες 
  stock[i]=stock[i].iloc[:,:5]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [131]:
stock

{'ABBV':                   Open        High         Low       Close   Adj Close
 Date                                                                  
 2021-05-03  112.400002  115.099998  111.589996  114.680000  108.952454
 2021-05-04  114.620003  115.940002  113.050003  113.900002  108.211418
 2021-05-05  114.309998  116.279999  113.949997  115.779999  109.997520
 2021-05-06  115.000000  116.180000  114.160004  116.080002  110.282524
 2021-05-07  116.379997  116.930000  114.959999  115.750000  109.969025
 ...                ...         ...         ...         ...         ...
 2022-05-23  151.649994  152.000000  147.190002  148.029999  146.658188
 2022-05-24  147.029999  151.350006  146.610001  149.110001  147.728180
 2022-05-25  150.360001  152.990005  149.789993  151.960007  150.551773
 2022-05-26  151.630005  153.070007  150.389999  150.570007  149.174652
 2022-05-27  149.520004  150.050003  147.750000  150.000000  148.609924
 
 [272 rows x 5 columns],
 'AEP':                   Ope

In [132]:
from datetime import datetime as dt 
for i in assets:
  stock[i].index= pd.to_datetime(stock[i].index)
print(len(stock['ABBV'].loc[(stock['ABBV'].index > '2022-04-27') & (stock['ABBV'].index <= '2022-05-27')]))

22


In [133]:
for j in assets:
    df=stock[j] #all the days we need so we can fix the columns 
    # print('to df ftanei mexri ',stock[j].iloc[:-61].tail(1))
    # print(df)
    #τώρα θα προσθέσουμε τις καινούριες στήλες με τα features μας 
    df['(open2-open1)/open2']= (df['Open'].shift(2)-df['Open'].shift(1))/df['Open'].shift(2)
    df['(high2-high1)/high2']= (df['High'].shift(2)-df['High'].shift(1))/df['High'].shift(2)
    df['(low2-low1)/low2']= (df['Low'].shift(2)-df['Low'].shift(1))/df['Low'].shift(2)
    df['(Adj Close2-Adj Close1)/Adj Close2']=(df['Adj Close'].shift(2)-df['Adj Close'].shift(1))/df['Adj Close'].shift(2)
    df['av_3']=df['Adj Close'].shift(1).rolling(window=3).mean().pct_change()
    df['av_6']=df['Adj Close'].shift(1).rolling(window=6).mean().pct_change()
    df['av_10']=df['Adj Close'].shift(1).rolling(window=10).mean().pct_change()
    df['av_14']=df['Adj Close'].shift(1).rolling(window=14).mean().pct_change()
    # df['av_14']=df['Adj Close'].shift(1).rolling(window=14).mean().pct_change()
    df['high1-low1/high1']=(df['High'].shift(1)-df['Low'].shift(1))/df['High'].shift(1)
    df['close1-open1/close1']=(df['Close'].shift(1)-df['Open'].shift(1))/df['Close'].shift(1)
    df['RSI']= ta.RSI(np.array(df['Adj Close'].shift(1)),timeperiod=6)/100
    df['ADX'] = ta.ADX(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1), timeperiod=6)/100
    df['RSI_14']= ta.RSI(np.array(df['Adj Close'].shift(1)),timeperiod=14)/100
    df['ADX_14'] = ta.ADX(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1), timeperiod=14)/100
    df['TRIX_14'] =ta.TRIX(df['Adj Close'].shift(1),timeperiod=14)
    df['TRIX_10'] =ta.TRIX(df['Adj Close'].shift(1),timeperiod=10)
    df['WILLR']= ta.WILLR(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1),timeperiod=14)/100
    df['std_dev']=df['Adj Close'].rolling(window=10).std().shift(1)
    df['dist_from_mean_10'] = (df['Adj Close'].shift(1) - (df['Adj Close'].shift(1).rolling(window=10).mean()))
    df['dist_from_mean_6'] = df['Adj Close'].shift(1) - (df['Adj Close'].shift(1).rolling(window=6).mean())
    df['MACD']= ta.MACD(df['Adj Close'].shift(1),fastperiod=5,slowperiod=14, signalperiod=8)[0]
    df['PPO']= ta.PPO(df['Adj Close'].shift(1), fastperiod= 5,slowperiod=14,matype=0)
    df['CMO_14']= ta.CMO(df['Adj Close'].shift(1),timeperiod=14)
    df = df.iloc[1: , :]
    df['absolute rank']= jk_feature[j].astype(int)
    df['Rank']= rd3[j]
    df1=df.dropna()
    stock[j]=df1.copy()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [134]:
#for target in 4 april I give the rank of 
for i in assets:
  print(stock[i].loc[stock[i].index=='2022-04-27']['Rank'])

Date
2022-04-27    1
Name: Rank, dtype: int64
Date
2022-04-27    1
Name: Rank, dtype: int64
Date
2022-04-27    2
Name: Rank, dtype: int64
Date
2022-04-27    4
Name: Rank, dtype: int64
Date
2022-04-27    5
Name: Rank, dtype: int64
Date
2022-04-27    2
Name: Rank, dtype: int64
Date
2022-04-27    5
Name: Rank, dtype: int64
Date
2022-04-27    4
Name: Rank, dtype: int64
Date
2022-04-27    4
Name: Rank, dtype: int64
Date
2022-04-27    4
Name: Rank, dtype: int64
Date
2022-04-27    2
Name: Rank, dtype: int64
Date
2022-04-27    3
Name: Rank, dtype: int64
Date
2022-04-27    1
Name: Rank, dtype: int64
Date
2022-04-27    3
Name: Rank, dtype: int64
Date
2022-04-27    3
Name: Rank, dtype: int64
Date
2022-04-27    4
Name: Rank, dtype: int64
Date
2022-04-27    2
Name: Rank, dtype: int64
Date
2022-04-27    5
Name: Rank, dtype: int64
Date
2022-04-27    2
Name: Rank, dtype: int64
Date
2022-04-27    2
Name: Rank, dtype: int64
Date
2022-04-27    5
Name: Rank, dtype: int64
Date
2022-04-27    1
Name: Rank, d

#####Logistic Regression (baseline)
***

In [135]:
from sklearn.linear_model import LogisticRegression
baseline= LogisticRegression()
#---predictions---#
prob1,first,last=predictions(baseline,assets,stock,29)
#---probabilities---#
mask = ((stock['ABBV'].index.values >= first.values) & (stock['ABBV'].index.values <= last.values))
prob1['date']=stock['ABBV'].loc[mask].index
prob1_final=pd.DataFrame.from_dict(prob1)
prob1_final.set_index('date')
RPS1= RPS_data(prob1_final,rd)
RPS1= create_RPS(prob1_final,RPS1)
RPS1
# print(RPS1.mean(axis=1).mean())

Date
2022-04-26    1
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-04-27'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
[0.12087518 0.21809012 0.43898516 0.10762836 0.11442119]
Date
2022-04-26    1
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-04-27'], dtype='datetime64[ns]', name='Date', freq=None)


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


this is last for prediction DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
[0.16619811 0.27031093 0.14839387 0.18320972 0.23188737]
Date
2022-04-26    1
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-04-27'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
[0.16923307 0.1797294  0.21790013 0.30573937 0.12739804]
Date
2022-04-26    4
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-04-27'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
[0.25391329 

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


this is the first day of predictions DatetimeIndex(['2022-04-27'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
[0.26595211 0.22938633 0.08383496 0.08704985 0.33377676]
Date
2022-04-26    5
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-04-27'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
[0.05030168 0.05765439 0.24987945 0.5427806  0.09938388]
Date
2022-04-26    5
Name: Rank, dtype: int64


...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV        AEP         AIZ       ALLE       AMAT  \
Date                                                                  
2022-05-26  151.076859  94.338966  172.290344  98.400002  94.777618   

                   AMP        AMZN         AVB        AVY         AXP  ...  \
Date                                                                   ...   
2022-05-26  242.633484  107.400002  195.470001  163.45636  141.659866  ...   

                  PRU   PYPL          RE        ROL       ROST        UNH  \
Date                                                                        
2022-05-26  96.546379  71.82  278.745331  34.418652  72.525093  506.84964   

                   URI           V        VRSK        XOM  
Date                                                       
2022-05-26  247.990005  197.769714  168.658813  90.619514  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ       ALLE       AMAT  \
Date                        

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob teleutaio :                   ABBV        AEP         AIZ       ALLE       AMAT  \
Date                                                                  
2022-05-27  152.711563  94.626709  170.169083  98.389999  91.705482   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-05-27  238.890869  108.919998  195.080002  163.336853  138.949997  ...   

                  PRU       PYPL          RE        ROL       ROST  \
Date                                                                 
2022-05-27  95.212051  71.470001  276.866699  34.877438  71.558487   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-05-27  514.096924  241.070007  199.147278  171.034866  87.272049  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP        AIZ        ALLE        AMAT  \

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-04-27,0.223226,0.254234,0.132360,0.129973,0.208425,0.095301,0.171814,0.103065,0.125651,0.169688,...,0.109725,0.318319,0.044237,0.273415,0.330763,0.249746,0.254251,0.081412,0.169016,0.157369
2022-04-28,0.117595,0.142169,0.208326,0.076620,0.148024,0.078191,0.114995,0.153643,0.184723,0.142152,...,0.145387,0.315425,0.034698,0.278732,0.301670,0.232861,0.225989,0.086020,0.222599,0.157054
2022-04-29,0.027964,0.113280,0.268007,0.331009,0.183361,0.239674,0.146439,0.096479,0.367447,0.142899,...,0.081626,0.179712,0.116256,0.170760,0.216781,0.077942,0.199860,0.085655,0.282282,0.133234
2022-05-02,0.112160,0.254419,0.131767,0.096100,0.153641,0.033197,0.097567,0.159739,0.192929,0.099946,...,0.187468,0.308456,0.049273,0.271517,0.107972,0.239667,0.114073,0.133786,0.190906,0.293580
2022-05-03,0.283861,0.096214,0.251193,0.308868,0.217809,0.120709,0.392055,0.372358,0.144246,0.125407,...,0.193875,0.136721,0.124243,0.196450,0.076605,0.040221,0.090024,0.080146,0.068940,0.137973
2022-05-04,0.252946,0.133344,0.129524,0.076575,0.148855,0.118711,0.112666,0.381547,0.147333,0.148101,...,0.138433,0.131811,0.279370,0.093942,0.203242,0.215512,0.162967,0.103980,0.048931,0.221251
2022-05-05,0.313795,0.156909,0.157608,0.108788,0.138520,0.079745,0.288028,0.391537,0.018772,0.075205,...,0.056284,0.356547,0.067902,0.103322,0.165660,0.183133,0.395907,0.126370,0.058182,0.077134
2022-05-06,0.322083,0.150629,0.062884,0.340797,0.239271,0.106652,0.134502,0.379394,0.111848,0.108657,...,0.064314,0.144609,0.272921,0.157706,0.364972,0.175265,0.034997,0.146003,0.237577,0.082831
2022-05-09,0.061958,0.084792,0.144508,0.098764,0.261453,0.132315,0.264540,0.182097,0.129790,0.264009,...,0.126507,0.100880,0.149288,0.097397,0.139334,0.330675,0.181889,0.260301,0.046702,0.170864


In [136]:
print(RPS1.mean(axis=1).mean())

0.16597893901434926


In [137]:
print(RPS1.mean(axis=1).head(1))

date
2022-04-27    0.168772
dtype: float64


In [143]:
#save results 
#save results 
RPS1.to_excel('2_May_27_May-log_reg(baseline)-real_data.xlsx')
prob1_final.set_index('date',inplace=True)
prob1_final.to_excel('2_May_27_May-PROBABILITIES-log_reg(baseline)-real_data.xlsx')

#####After feature importances
***

In [138]:
stock1=stock.copy()
# print(stock2)
for j in assets:
  stock1[j]=stock1[j].drop(['absolute rank','CMO_14','RSI','RSI_14','PPO','MACD','TRIX_14','TRIX_10','dist_from_mean_6','WILLR','dist_from_mean_10','std_dev','(Adj Close2-Adj Close1)/Adj Close2','ADX_14'],axis=1)

In [139]:
from sklearn.linear_model import LogisticRegression
model1= LogisticRegression(max_iter=500)
#---predictions---#
prob2 ,first,last =predictions(model1,assets,stock1,15)
#---probabilities---#
mask = ((stock['ABBV'].index.values >= first.values) & (stock['ABBV'].index.values <= last.values))
prob2['date']=stock['ABBV'].loc[mask].index
prob2_final=pd.DataFrame.from_dict(prob2)
prob2_final.set_index('date')
RPS2= RPS_data(prob2_final,rd)
RPS2= create_RPS(prob2_final,RPS2)
RPS2
# print(RPS2.mean(axis=1).mean())

Date
2022-04-26    1
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-04-27'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
[0.15543547 0.17372371 0.20601709 0.22469415 0.24012958]
Date
2022-04-26    1
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-04-27'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
[0.16436683 0.22672546 0.16974258 0.24255365 0.19661147]
Date
2022-04-26    1
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-04-27'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction Datetim

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kerne

prob :                    ABBV         AEP         AIZ        ALLE       AMAT  \
Date                                                                    
2022-04-28  148.609924  102.762794  180.665848  113.080322  119.17524   

                   AMP      AMZN         AVB         AVY         AXP  ...  \
Date                                                                  ...   
2022-04-28  277.202972  115.1465  209.208252  172.161026  168.967667  ...   

                  PRU       PYPL          RE        ROL       ROST  \
Date                                                                 
2022-04-28  105.03669  85.209999  283.079163  36.513096  86.678146   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-04-28  503.727386  297.459991  212.503616  178.010666  96.650909  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP        AIZ        ALLE        AMAT  \
Date    

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV         AEP        AIZ        ALLE        AMAT  \
Date                                                                    
2022-04-29  146.004288  101.234802  175.96521  111.197975  116.990822   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-04-29  274.993225  120.209503  206.282059  171.862244  168.190582  ...   

                   PRU       PYPL          RE        ROL       ROST  \
Date                                                                  
2022-04-29  105.016922  85.209999  280.802887  35.366138  84.404716   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-04-29  493.466309  298.179993  211.794876  174.294022  95.076218  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ        ALLE        AMAT  \

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV        AEP         AIZ       ALLE       AMAT  \
Date                                                                  
2022-05-26  151.076859  94.338966  172.290344  98.400002  94.777618   

                   AMP        AMZN         AVB        AVY         AXP  ...  \
Date                                                                   ...   
2022-05-26  242.633484  107.400002  195.470001  163.45636  141.659866  ...   

                  PRU   PYPL          RE        ROL       ROST        UNH  \
Date                                                                        
2022-05-26  96.546379  71.82  278.745331  34.418652  72.525093  506.84964   

                   URI           V        VRSK        XOM  
Date                                                       
2022-05-26  247.990005  197.769714  168.658813  90.619514  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ       ALLE       AMAT  \
Date                        

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob teleutaio :                   ABBV        AEP         AIZ       ALLE       AMAT  \
Date                                                                  
2022-05-27  152.711563  94.626709  170.169083  98.389999  91.705482   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-05-27  238.890869  108.919998  195.080002  163.336853  138.949997  ...   

                  PRU       PYPL          RE        ROL       ROST  \
Date                                                                 
2022-05-27  95.212051  71.470001  276.866699  34.877438  71.558487   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-05-27  514.096924  241.070007  199.147278  171.034866  87.272049  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP        AIZ        ALLE        AMAT  \

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-04-27,0.289653,0.256085,0.120749,0.126622,0.202212,0.138393,0.270284,0.116087,0.124581,0.133362,...,0.071564,0.283869,0.071828,0.237776,0.220089,0.280376,0.235677,0.078548,0.120212,0.270429
2022-04-28,0.151791,0.122272,0.277947,0.108789,0.146273,0.072781,0.276188,0.117053,0.124421,0.131824,...,0.119293,0.277244,0.072018,0.241219,0.220360,0.280710,0.237712,0.129360,0.251655,0.163008
2022-04-29,0.082153,0.109413,0.248469,0.271679,0.222329,0.272848,0.270686,0.116175,0.272431,0.109607,...,0.070270,0.140392,0.131727,0.130934,0.216564,0.067577,0.127970,0.078980,0.251920,0.178173
2022-05-02,0.153062,0.259201,0.121140,0.073820,0.139207,0.073061,0.284061,0.116180,0.125018,0.133652,...,0.247386,0.300402,0.071382,0.248511,0.113705,0.277462,0.139478,0.127959,0.120502,0.276696
2022-05-03,0.190782,0.108061,0.249754,0.228560,0.221711,0.137175,0.184818,0.245274,0.123072,0.109196,...,0.245325,0.145360,0.113781,0.224909,0.089954,0.134340,0.234188,0.078513,0.116231,0.180742
2022-05-04,0.291991,0.124572,0.107761,0.072522,0.142717,0.097451,0.281532,0.244283,0.122790,0.112190,...,0.120415,0.134026,0.217505,0.083278,0.216651,0.104728,0.236058,0.078778,0.058038,0.164217
2022-05-05,0.190276,0.125356,0.108001,0.124298,0.141057,0.074060,0.184366,0.255383,0.067286,0.076250,...,0.067229,0.292638,0.072323,0.113234,0.214590,0.102818,0.233945,0.111760,0.113082,0.178077
2022-05-06,0.191699,0.126286,0.121381,0.231194,0.220557,0.142157,0.091723,0.245061,0.108777,0.110143,...,0.067561,0.133744,0.272707,0.133719,0.253911,0.100832,0.208096,0.129368,0.253926,0.183072
2022-05-09,0.154007,0.082750,0.105899,0.120294,0.219397,0.143070,0.182720,0.113020,0.109796,0.239925,...,0.115405,0.149634,0.113455,0.111681,0.131123,0.210225,0.107833,0.226953,0.054130,0.100635


In [140]:
print(RPS2.mean(axis=1).mean())

0.15869891393298446


In [141]:
print(RPS2.mean(axis=1))

date
2022-04-27    0.164041
2022-04-28    0.165113
2022-04-29    0.158213
2022-05-02    0.169081
2022-05-03    0.146651
2022-05-04    0.162371
2022-05-05    0.149891
2022-05-06    0.164491
2022-05-09    0.156991
2022-05-10    0.148978
2022-05-11    0.152995
2022-05-12    0.163172
2022-05-13    0.165942
2022-05-16    0.153027
2022-05-17    0.166257
2022-05-18    0.150784
2022-05-19    0.158588
2022-05-20    0.165763
2022-05-23    0.165611
2022-05-24    0.150628
2022-05-25    0.151477
2022-05-26    0.161669
2022-05-27    0.158342
dtype: float64


In [145]:
#save results 
RPS2.to_excel('2_May_27_May-log_reg(with_feature_imp)-real_data.xlsx')
prob2_final.set_index('date',inplace=True)
prob2_final.to_excel('2_May_27_May-PROBABILITIES-log_reg(with_feature_imp)-real_data.xlsx')